In [1]:
import functools as fn
import re

import numpy as np
import pandas as pd
import spacy
from bs4 import BeautifulSoup

In [2]:
# text processing functions
nlp = spacy.load('en_core_web_md', disable=['tagger', 'ner'])


def doc_lemmas(doc):
    """list of lemmas from a doc(ument)"""
    return [t.lemma_.lower() for t in doc if not (t.is_stop or t.is_punct)]


def doc_lemma_str(doc):
    """string of lemmas from a doc(ument)"""
    return ' '.join(doc_lemmas(doc))


def lemma_str(text):
    """string of lemmas from text"""
    return doc_lemma_str(nlp(text))


def no_html(html_text):
    """text content from html"""
    return BeautifulSoup(html_text, 'lxml').text


def normalized_lines(text):
    """normalized text with no more than one consecutive newline character"""
    lines = text.splitlines()
    non_empty = [x for x in lines if x != '']
    return '\n'.join(non_empty)


def value_is_nan(x):
    """Returns true if x is NaN; safe for x of unknown type"""
    return isinstance(x, float) and np.isnan(x)

## SOC Codes

The U.S. Bureau of Labor Statistics (BLS) defines 867 [Standard Occupational Classification (SOC) codes](https://www.bls.gov/soc/) (in the 2018 scheme). These are the primary targets of the classification task. As part of the SOC system, the BLS also publishes a set of "[direct match titles](https://www.bls.gov/soc/2018/soc_2018_direct_match_title_file.xlsx)" that unambiguously indicate a specific SOC code. I use these titles to label any job ads with matching titles, which I then use to benchmark the models that I develop.

The [O\*NET SOC 2019 taxonomy](https://www.onetcenter.org/taxonomy.html) supplements the BLS SOC 2018 system with more detailed descriptions of tasks performed in each occupation, and a slightly more fine-grained set of 1,016 occupations. I use this to create more fleshed-out occupation reference documents for comparisons with job ads.

### TODO: O*NET Taxonomy

Incorporate the [O*NET taxonomy](https://www.onetcenter.org/taxonomy.html) and other [O*NET databases](https://www.onetcenter.org/database.html), including:
- Work activities (which apparently are different from tasks?)
- Knowledge/skills/abilities, education/experience/training
- Interests, work values, work styles
- Technology skills & tools
- Context
- Related occupations (should be close to each other in embedded space)

In [3]:
# read SOC hierarchical structure
soc_path = '../source_data/onet_soc/SOC_Structure.xlsx'
columns = ['soc_major', 'soc_minor', 'soc_broad', 'soc_detail', 'soc_onet', 'title']
occupations = pd.read_excel(soc_path, skiprows=np.arange(4), names=columns, header=None)

In [4]:
# fill o*net codes with detail codes
soc_onet = occupations['soc_onet'].values
for i, detail in enumerate(occupations['soc_detail']):
    if value_is_nan(soc_onet[i]) and not value_is_nan(detail):
        soc_onet[i] = detail + '.00'

occupations['soc_onet'] = soc_onet

# fill classification hierarchy
occupations['soc_major'] = occupations['soc_major'].ffill()

fill_cols = ['soc_major', 'soc_minor', 'soc_broad', 'soc_detail']
for i in range(1, len(fill_cols)):
    group = fill_cols[:i]
    target = fill_cols[i]
    occupations[target] = occupations.groupby(group)[target].ffill()

In [5]:
# focus on most detailed level of occupations (which have definitions)
onet_occs = occupations[occupations['soc_onet'].notna()]
onet_occs.head(10)

,soc_major,soc_minor,soc_broad,soc_detail,soc_onet,title
3,11-0000,11-1000,11-1010,11-1011,11-1011.00,Chief Executives
4,11-0000,11-1000,11-1010,11-1011,11-1011.03,Chief Sustainability Officers
6,11-0000,11-1000,11-1020,11-1021,11-1021.00,General and Operations Managers
8,11-0000,11-1000,11-1030,11-1031,11-1031.00,Legislators
11,11-0000,11-2000,11-2010,11-2011,11-2011.00,Advertising and Promotions Managers
13,11-0000,11-2000,11-2020,11-2021,11-2021.00,Marketing Managers
14,11-0000,11-2000,11-2020,11-2022,11-2022.00,Sales Managers
16,11-0000,11-2000,11-2030,11-2032,11-2032.00,Public Relations Managers
17,11-0000,11-2000,11-2030,11-2033,11-2033.00,Fundraising Managers
20,11-0000,11-3000,11-3010,11-3012,11-3012.00,Administrative Services Managers


In [6]:
# # TODO: consider re-inserting higher-level aggregations
# def int_agg(soc_code):
#     """
#     Intermediate aggregation level of an SOC code
#     See soc_2018_manual.pdf, pg. 15, https://www.bls.gov/soc/2018/soc_2018_manual.pdf
#     """
#     major_group = int(soc_code[:2])
#     for i, bound in enumerate([13, 19, 27, 29, 39, 41, 43, 45, 47, 49, 51, 53, 55]):
#         if major_group <= bound:
#             return i + 1
#     return None


# def high_agg(soc_code):
#     """
#     High-level aggregation level of an SOC code
#     See soc_2018_manual.pdf, pg. 16, https://www.bls.gov/soc/2018/soc_2018_manual.pdf
#     """
#     major_group = int(soc_code[:2])
#     for i, bound in enumerate([29, 39, 43, 49, 53, 55]):
#         if major_group <= bound:
#             return i + 1
#     return None


# bls_soc['int_agg'] = bls_soc['major'].apply(int_agg)
# bls_soc['high_agg'] = bls_soc['major'].apply(high_agg)

In [7]:
# read SOC definitions
soc_path = '../source_data/onet_soc/2019_Occupations.xlsx'
columns = ['soc_onet', 'title', 'definition']
soc_defs = pd.read_excel(soc_path, skiprows=np.arange(4),
                         names=columns, header=None)

In [8]:
# combine SOC classification hierarchy with definitions
onet_occs = onet_occs.set_index('soc_onet')
soc_defs = soc_defs.set_index('soc_onet')
onet_occs = pd.concat([onet_occs, soc_defs['definition']], axis=1)

In [9]:
onet_occs.head(10)

,soc_major,soc_minor,soc_broad,soc_detail,title,definition
soc_onet,,,,,,
11-1011.00,11-0000,11-1000,11-1010,11-1011,Chief Executives,Determine and formulate policies and provide o...
11-1011.03,11-0000,11-1000,11-1010,11-1011,Chief Sustainability Officers,"Communicate and coordinate with management, sh..."
11-1021.00,11-0000,11-1000,11-1020,11-1021,General and Operations Managers,"Plan, direct, or coordinate the operations of ..."
11-1031.00,11-0000,11-1000,11-1030,11-1031,Legislators,"Develop, introduce, or enact laws and statutes..."
11-2011.00,11-0000,11-2000,11-2010,11-2011,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising polici..."
11-2021.00,11-0000,11-2000,11-2020,11-2021,Marketing Managers,"Plan, direct, or coordinate marketing policies..."
11-2022.00,11-0000,11-2000,11-2020,11-2022,Sales Managers,"Plan, direct, or coordinate the actual distrib..."
11-2032.00,11-0000,11-2000,11-2030,11-2032,Public Relations Managers,"Plan, direct, or coordinate activities designe..."
11-2033.00,11-0000,11-2000,11-2030,11-2033,Fundraising Managers,"Plan, direct, or coordinate activities to soli..."


In [10]:
# omit "Excludes..." statements from definitions
# see (https://www.bls.gov/soc/2018/soc_2018_manual.pdf), pg. 13
def without_exclude(text):
    """Returns text with 'Excludes' statements removed"""
    doc = nlp(text)
    sents = [s.string.strip()
             for s in doc.sents if not s.string.startswith('Exclud')]
    return ' '.join(sents)


onet_occs['definition'] = onet_occs['definition'].apply(without_exclude)

In [11]:
# read occupation tasks
tasks_path = '../source_data/onet_soc/2019_Tasks.xlsx'
tasks = pd.read_excel(tasks_path, skiprows=np.arange(4), usecols=[0, 2],
                      names=['soc_onet', 'task'], header=None)

In [12]:
# concatenate tasks for the same occupation
tasks = tasks.groupby('soc_onet')['task']
tasks = tasks.apply(lambda x: '\n'.join(x))


# concatenate tasks with occupation definitions
def merged_soc_def(soc_onet):
    d = onet_occs.loc[soc_onet, 'definition']
    try:
        return f'{d}\n{tasks[soc_onet]}'
    except:
        return d


onet_occs['definition'] = [merged_soc_def(i) for i in onet_occs.index]

In [13]:
# re-name columns for export
onet_occs = onet_occs.reset_index()
columns = onet_occs.columns.tolist()
columns[0] = 'soc_onet'
onet_occs.columns = columns

In [14]:
# lemmatize description text
onet_occs['def_lemma'] = onet_occs['definition'].apply(lemma_str)

In [15]:
# read SOC match titles
titles_path = '../source_data/bls_soc/soc_2018_direct_match_title_file.xlsx'
match_titles = pd.read_excel(titles_path, skiprows=np.arange(8), usecols=[0, 2],
                             names=['soc_detail', 'title'], header=None)

In [16]:
match_titles.head(10)

,soc_detail,title
0,11-1011,Admiral
1,11-1011,CEO
2,11-1011,Chief Executive Officer
3,11-1011,Chief Financial Officer
4,11-1011,Chief Operating Officer
5,11-1011,Chief Sustainability Officer
6,11-1011,Commissioner of Internal Revenue
7,11-1011,COO
8,11-1011,County Commissioner
9,11-1011,Government Service Executive


I identify jobs with a "direct match" to an SOC code (occupation) as any job where the job title *contains* a title from the BLS direct match file. This amounts to including any more specialized jobs under the banner of more general occupations. It also tends to be consistent with two significant principles of the SOC system:

- Trainees for an occupation are counted in the occupation they are training for. So a "Property Manager in Training (MIT)" is included in "Property, Real Estate, and Community Association Managers" (11-9141), which includes "Property Manager" as a direct match title.
- Direct managers of individual contributors in an occupation are often counted in the occupation of the staff they manage.

I remove some titles from the direct match title file because they match too many unrelated jobs when using a containment test (as opposed to matching full job titles):

- "Operations Analyst" ("Operations Research Analysts", 15-2031)
- "Coach" ("Coaches and Scounts", 27-2022)
- "Acrobat" ("Entertainers and Performers, Sports and Related Workers, All Other", 27-2099)
- "Author" ("Writers and Authors", 27-3043)
- "OT" ("Occupational Therapists", 29-1122)
- "RRT" ("Respiratory Therapists", 29-1126)
- "PT" ("Physical therapists", 29-1123)
- "Blaster" ("Explosives Workers, Ordnance Handling Experts, and Blasters", 47-5032)

I also remove all military occupations (55-*). Specific military job openings are not advertised on public job boards.

In [17]:
# ignore military occupations
onet_occs = onet_occs[onet_occs['soc_major'] != '55-0000']
match_titles = match_titles[~match_titles['soc_detail'].str.startswith('55-')]

# other exclusions
exclude = [
    'Operations Analyst',
    'Coach',
    'Acrobat',
    'Author',
    'OT',
    'RRT',
    'PT',
    'Blaster'
]
match_titles = match_titles[~match_titles['title'].isin(exclude)]

In [18]:
# save wrangled SOC codes
soc_path = '../processed_data/occupations.feather'
onet_occs.to_feather(soc_path)

match_titles_path = '../processed_data/soc_direct_match_titles.feather'
match_titles.reset_index().to_feather(match_titles_path)

## Indeed Job Descriptions

[This data set was posted by PromptCloud and Datastock to Kaggle](https://www.kaggle.com/promptcloud/us-job-data-careerbuildercom-2019), apparently in an effort to market their web scraping services.

The data are 30,002 U.S. job ads scraped from Indeed.com between 2019-08-01 and 2019-10-31.

Since the focus of this project is labeling job descriptions with SOC codes, I will only use the `Job Title` and `Job Description` columns. The other columns are either empty, not relevant, or we'd like the model to work well without that information because it may not be available in other contexts.

### TODO

- Include location to see if it improves results.
- Include company name or description to see if it improves results. (Company description is often included in job ad body text.)

In [19]:
indeed_url = '../source_data/indeed/marketing_sample_for_trulia_com-real_estate__20190901_20191031__30k_data.csv'
jobs = pd.read_csv(indeed_url, header=0, names=['title', 'description'],
                   usecols=[0, 1])
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30002 entries, 0 to 30001
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        30002 non-null  object
 1   description  30002 non-null  object
dtypes: object(2)
memory usage: 468.9+ KB


In [20]:
# clean-up job text
def clean_description(text):
    return normalized_lines(no_html(text))


jobs['description'] = [clean_description(d) for d in jobs['description']]
jobs['desc_lemma'] = jobs['description'].apply(lemma_str)

In [21]:
jobs.head(10)

,title,description,desc_lemma
0,shift manager,WE ARE LOOKING FOR TOP PERFORMERS TO GROW WITH...,looking performers grow \n del taco need peopl...
1,operations support manager,JOB PURPOSE: This position is responsible for ...,job purpose position responsible provide opera...
2,senior product manager - data,Product Manager - Unique opportunity with a mo...,product manager unique opportunity mobile mark...
3,part-time office concierge,"Coldwell Banker in Festus, Missouri is looking...",coldwell banker festus missouri look customer ...
4,print & marketing associate,Description\nPosition Summary:\nThe Copy and P...,description \n position summary \n copy print ...
5,cyber it risk & strategy senior consultant,As a Cyber Risk Senior Consultant for Deloitte...,cyber risk senior consultant deloitte financia...
6,"sales associate, retail part time",GNC is looking for dynamic sales associates th...,gnc look dynamic sale associate live lifestyle...
7,home lending branch manager-spokane,JOB PURPOSE\nResponsible for residential loan ...,job purpose \n responsible residential loan pr...
8,property manager in training (mit),"Overview\nJob Location\nDurham, NC - South Squ...",overview \n job location \n durham nc south sq...
9,"compliance specialist, marketing, advertising ...",About Green Thumb Industries:\nGreen Thumb Ind...,green thumb industries \n green thumb industri...


In [42]:
match_titles['title'] = match_titles['title'].str.lower()
job_titles = jobs['title'].str.lower()

def word_match(search_text, text):
    """True if text contains whole words in search_text; False otherwise"""
    match = re.search(r'\b' + search_text + r'\b', text)
    return match is not None


def direct_match_soc_codes():
    """array of SOC codes for jobs with direct title matches (or pd.NA in the absence of a match)"""
    n = len(jobs)
    job_direct_soc = np.full(n, pd.NA)
    match_count = np.full(n, 0)

    for soc_detail, title in match_titles.itertuples(index=False):
        try:
            # find jobs matching title
            match = job_titles.apply(fn.partial(word_match, title))
            job_direct_soc = np.where(match, soc_detail, job_direct_soc)
            match_count += match.astype(int)  # track match collisions
        except:
            pass

    # ignore any SOCs with match collisions, which make the SOC ambiguous
    job_direct_soc = np.where(match_count > 1, pd.NA, job_direct_soc)

    return job_direct_soc

In [43]:
jobs['soc_detail'] = direct_match_soc_codes()

In [44]:
dm_jobs = jobs[jobs['soc_detail'].notna()]  # direct matched jobs
n_dm = len(dm_jobs)
n_jobs = len(jobs)
print(f'{n_dm} jobs out of {n_jobs} ({n_dm / n_jobs:.0%}) could be directly matched by title.')

n_dmsoc = dm_jobs['soc_detail'].nunique()
n_soc = onet_occs['soc_detail'].nunique()
print(f'{n_dmsoc} occupations out of {n_soc} ({n_dmsoc / n_soc:.0%}) are represented in the directly matched jobs.')

3163 jobs out of 30002 (11%) could be directly matched by title.
167 occupations out of 848 (20%) are represented in the directly matched jobs.


In [45]:
# feather format doesn't support pd.NA
jobs['soc_detail'] = ['' if pd.isna(soc) else soc for soc in jobs['soc_detail']]

# save wrangled job ads
jobs_path = '../processed_data/job_ads.feather'
jobs.to_feather(jobs_path)

### TODO 
## CareerBuilder Job Descriptions

[This data set was posted by PromptCloud and Datastock to Kaggle](https://www.kaggle.com/promptcloud/us-job-data-careerbuildercom-2019), apparently in an effort to market their web scraping services.

This data is in LD-JSON format, which Pandas does not read. the `pytablereader` package is also not able to read this file due to a character encoding issue.

In [ ]:
# cb_url = '../source_data/careerbuilder/marketing_sample_for_careerbuilder_com-jobs__20190901_20191231__30k_data.ldjson'
# cd_df = pd.read_json(cb_url)

# import pytablereader as ptr
# cd_df = ptr.JsonLinesTableFileLoader(cb_url).load()